In [12]:
import os
import django


# Set up Django settings
os.environ["DJANGO_SETTINGS_MODULE"] = "memoria.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

# Now you can import your models and work with them
from core.models import Noticia

# Example: Query all objects
noticia = Noticia.objects.get(pk=22)

In [14]:
noticia.archivo_url

'http://web.archive.org/web/20250226141907/https://ladiaria.com.uy/politica/articulo/2025/2/la-coalicion-maneja-pedirle-al-menos-cuatro-cargos-mas-al-fa-la-caru-salto-grande-la-unasev-y-el-inumet/'

In [11]:
noticia.get_title_from_meta_tags()

Title: la diaria, Image: https://ladiaria.com.uy/static/meta/la-diaria-1000x1000.png


In [ ]:
from bs4 import BeautifulSoup
import requests

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/90.0.4430.93 Safari/537.36"
    )
}

response = requests.get(noticia.enlace, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# Extract Open Graph meta tags
og_title = soup.find("meta", property="og:title")
og_image = soup.find("meta", property="og:image")

In [17]:
r = requests.options(noticia.enlace, headers=headers)
print(r.text)

<!doctype html>

<!-- Legacy browsers support  -->
<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="es-UY"> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8" lang="es-UY"> <![endif]-->
<!--[if IE 8]>         <html class="no-js lt-ie9" lang="es-UY"> <![endif]-->
<!-- End legacy browsers support  -->
<html class="no-js" lang="es-UY">

  <head>
    <link rel="canonical" href="https://ladiaria.com.uy/usuarios/entrar/">
    
      <!-- Progressive Web App (PWA) -->
      <link rel="manifest" href="/static/manifest-ladiaria.json">
      <!-- End Progressive Web App (PWA) -->
    
    
      
      
        <!-- Google Tag Manager -->
        <script type="6c7fc3855227d095916f4250-text/javascript">
          (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
          new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
          j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
          'https://www.go

In [7]:
og_title

<meta content="la diaria" property="og:title"/>

In [ ]:
from core import archive

original_url = "https://ladiaria.com.uy/politica/articulo/2025/2/gobierno-electo-le-ofrecio-34-cargos-a-la-oposicion-y-la-coalicion-se-reunira-para-hacer-una-contrapropuesta/"


In [3]:
wayback_url = archive.get_wayback_latest_snapshot(original_url)
html = archive.fetch_wayback_snapshot(wayback_url)
details = archive.parse_wayback_details(html)

In [4]:
details

{'title': 'Gobierno electo le ofreció 34 cargos a la oposición y la coalición se reunirá para hacer una contrapropuesta | la diaria | Uruguay',
 'archive_date': '2025-02-25T00:54:47+00:00',
 'screenshot_url': 'http://web.archive.org/web/20250226133810im_/https://ladiaria.com.uy/media/photologue/photos/cache/cargos_RS261834_20250224_MVU-6892_article_main.jpg?r=UwRU3mYx0ByPEpPP',
 'archive_url': 'http://web.archive.org/web/20250226133810/https://ladiaria.com.uy/politica/articulo/2025/2/gobierno-electo-le-ofrecio-34-cargos-a-la-oposicion-y-la-coalicion-se-reunira-para-hacer-una-contrapropuesta/'}

In [13]:
result = check_wayback(article)
if not result:
    save_link = f"https://web.archive.org/save/{article}"
    print("No snapshot found. You can save it here:", save_link)
else:
    print("Snapshot found:", result)

Snapshot found: http://web.archive.org/web/20250226133810/https://ladiaria.com.uy/politica/articulo/2025/2/gobierno-electo-le-ofrecio-34-cargos-a-la-oposicion-y-la-coalicion-se-reunira-para-hacer-una-contrapropuesta/


In [5]:
from core import parse

markdwon = parse.parse_noticia_markdown(html, metadata["title"])


In [6]:
print(markdwon)

# Gobierno electo le ofreció 34 cargos a la oposición y la coalición se reunirá para hacer una contrapropuesta

**Fuente:** la diaria

**Autor:** No especificado

Este lunes, en la sede de la transición de gobierno, el edificio Plaza Alemania, el futuro secretario de Presidencia, Alejandro Sánchez, y el próximo prosecretario de Presidencia, Jorge Díaz, recibieron a los representantes de los partidos de la oposición para hacer oficial la propuesta de cargos. A la reunión asistieron el senador y excandidato Álvaro Delgado, por el Partido Nacional (PN), el senador y secretario general del Partido Colorado (PC), Andrés Ojeda, el exsenador Guido Manini Ríos, líder de Cabildo Abierto (CA), Pablo Mieres, líder del Partido Independiente, y el diputado Gustavo Salle, líder de Identidad Soberana. Este último, como ya había adelantado, informó que no aceptará cargos.

El gobierno electo ofreció a la oposición 34 cargos: seis en la Corte Electoral (cuatro ministros, un secretario letrado y un subd

In [7]:
google_flash = "openrouter/mistralai/mistral-saba"

In [25]:
dolphin = "openrouter/perplexity/r1-1776"

In [26]:
md2 = parse.parse_noticia_markdown(html, metadata["title"], dolphin)

In [27]:
print(md2)

```markdown
# Gobierno electo le ofreció 34 cargos a la oposición y la coalición se reunirá para hacer una contrapropuesta

## El senador blanco Álvaro Delgado adelantó que plantearán “diferentes responsabilidades, que no están consideradas hoy”, así como “algunos enfoques diferentes”

**Fuente:** la diaria  
**Publicado el:** 24 de febrero de 2025  

Este lunes, en la sede de la transición de gobierno, el edificio Plaza Alemania, el futuro secretario de Presidencia, Alejandro Sánchez, y el próximo prosecretario de Presidencia, Jorge Díaz, recibieron a los representantes de los partidos de la oposición para hacer oficial la propuesta de cargos. A la reunión asistieron el senador y excandidato Álvaro Delgado, por el Partido Nacional (PN), el senador y secretario general del Partido Colorado (PC), Andrés Ojeda, el exsenador Guido Manini Ríos, líder de Cabildo Abierto (CA), Pablo Mieres, líder del Partido Independiente, y el diputado Gustavo Salle, líder de Identidad Soberana. Este último

In [31]:
res_flash = parse.parse_noticia(markdwon, google_flash)
print(res_flash.model_dump())

{'titulo': 'Gobierno electo le ofreció 34 cargos a la oposición y la coalición se reunirá para hacer una contrapropuesta', 'fuente': 'la diaria', 'categoria': 'politica', 'autor': None, 'resumen': 'El gobierno electo ofreció 34 cargos a la oposición en una reunión en la sede de la transición de gobierno. Los representantes de los partidos de la oposición asistieron y discutieron la propuesta. La oposición se reunirá para hacer una contrapropuesta.', 'entidades': [{'nombre': 'Alejandro Sánchez', 'tipo': 'persona', 'sentimiento': 'neutral'}, {'nombre': 'Jorge Díaz', 'tipo': 'persona', 'sentimiento': 'neutral'}, {'nombre': 'Álvaro Delgado', 'tipo': 'persona', 'sentimiento': 'neutral'}, {'nombre': 'Andrés Ojeda', 'tipo': 'persona', 'sentimiento': 'neutral'}, {'nombre': 'Guido Manini Ríos', 'tipo': 'persona', 'sentimiento': 'neutral'}, {'nombre': 'Pablo Mieres', 'tipo': 'persona', 'sentimiento': 'neutral'}, {'nombre': 'Gustavo Salle', 'tipo': 'persona', 'sentimiento': 'neutral'}, {'nombre':

In [ ]:
res_o3 = parse.parse_noticia(markdown=markdwon)
print(res_o3.model_dump())

{'titulo': 'Gobierno electo le ofreció 34 cargos a la oposición y la coalición se reunirá para hacer una contrapropuesta', 'fuente': 'la diaria', 'categoria': None, 'autor': None, 'resumen': 'El senador blanco Álvaro Delgado adelantó que plantearán “diferentes responsabilidades, que no están consideradas hoy”, así como “algunos enfoques diferentes”', 'entidades': [{'nombre': 'Álvaro Delgado', 'tipo': 'persona', 'sentimiento': 'neutral'}, {'nombre': 'Alejandro Sánchez', 'tipo': 'persona', 'sentimiento': 'neutral'}, {'nombre': 'Jorge Díaz', 'tipo': 'persona', 'sentimiento': 'neutral'}, {'nombre': 'Andrés Ojeda', 'tipo': 'persona', 'sentimiento': 'neutral'}, {'nombre': 'Guido Manini Ríos', 'tipo': 'persona', 'sentimiento': 'neutral'}, {'nombre': 'Pablo Mieres', 'tipo': 'persona', 'sentimiento': 'neutral'}, {'nombre': 'Gustavo Salle', 'tipo': 'persona', 'sentimiento': 'neutral'}, {'nombre': 'Partido Nacional', 'tipo': 'organizacion', 'sentimiento': 'neutral'}, {'nombre': 'Partido Colorado'

In [ ]:
deep = "openrouter/deepseek/deepseek-r1"
res_deep = parse.parse_noticia(markdown=markdwon, model=deep)
print(res_deep.model_dump())

KeyboardInterrupt: 

In [ ]:
# pretty print
def pretty_print_articulo(obj):
    print(f"Titulo: {obj.titulo}")
    print(f"Fuente: {obj.fuente}")
    print(f"Categoria: {obj.categoria}")
    print(f"Resumen: {obj.resumen}")
    print(f"Cantidad de entidades: {len(obj.entidades)}")
    for entidad in obj.entidades:
        print(f"Entidad: {entidad}")

In [34]:
pretty_print_articulo(res_flash)

Titulo: Gobierno electo le ofreció 34 cargos a la oposición y la coalición se reunirá para hacer una contrapropuesta
Fuente: la diaria
Categoria: politica
Resumen: El gobierno electo ofreció 34 cargos a la oposición en una reunión en la sede de la transición de gobierno. Los representantes de los partidos de la oposición asistieron y discutieron la propuesta. La oposición se reunirá para hacer una contrapropuesta.
Cantidad de entidades: 35
Entidad: nombre='Alejandro Sánchez' tipo='persona' sentimiento='neutral'
Entidad: nombre='Jorge Díaz' tipo='persona' sentimiento='neutral'
Entidad: nombre='Álvaro Delgado' tipo='persona' sentimiento='neutral'
Entidad: nombre='Andrés Ojeda' tipo='persona' sentimiento='neutral'
Entidad: nombre='Guido Manini Ríos' tipo='persona' sentimiento='neutral'
Entidad: nombre='Pablo Mieres' tipo='persona' sentimiento='neutral'
Entidad: nombre='Gustavo Salle' tipo='persona' sentimiento='neutral'
Entidad: nombre='Partido Nacional' tipo='organizacion' sentimiento='n

In [21]:
pretty_print_articulo(res_o3)

Titulo: Gobierno electo le ofreció 34 cargos a la oposición y la coalición se reunirá para hacer una contrapropuesta
Fuente: la diaria
Categoria: None
Resumen: El senador blanco Álvaro Delgado adelantó que plantearán “diferentes responsabilidades, que no están consideradas hoy”, así como “algunos enfoques diferentes”
Cantidad de entidades: 11
Entidad: nombre='Álvaro Delgado' tipo='persona' sentimiento='neutral'
Entidad: nombre='Alejandro Sánchez' tipo='persona' sentimiento='neutral'
Entidad: nombre='Jorge Díaz' tipo='persona' sentimiento='neutral'
Entidad: nombre='Andrés Ojeda' tipo='persona' sentimiento='neutral'
Entidad: nombre='Guido Manini Ríos' tipo='persona' sentimiento='neutral'
Entidad: nombre='Pablo Mieres' tipo='persona' sentimiento='neutral'
Entidad: nombre='Gustavo Salle' tipo='persona' sentimiento='neutral'
Entidad: nombre='Partido Nacional' tipo='organizacion' sentimiento='neutral'
Entidad: nombre='Partido Colorado' tipo='organizacion' sentimiento='neutral'
Entidad: nombr